In [1]:
import sys
sys.path.append('/Users/avgalichin/Documents/kaggle/Jigsaw2022')
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer, RobertaConfig

from src.losses import MarginRankingLoss
from src.models import _initialize_hugginface_pair_model, BertBaseModel
# from src.models import RobertaModel
from src.dataset import Jigsaw22DatasetPaired

/Users/avgalichin/anaconda3/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.6.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [8]:
path = "/Users/avgalichin/Documents/kaggle/Jigsaw2022/Datasets/Initial/jigsaw22_validation_data_swap.csv"
data = pd.read_csv(path)
print(data.shape)
data.head()

(15410, 3)


,less_toxic,more_toxic,label
0,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...,This article sucks \n\nwoo woo wooooooo,1
1,Daphne Guinness \n\nTop of the mornin' my fav...,"""And yes, people should recognize that but the...",1
2,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu...",0
3,You seem to have sand in your vagina.\n\nMight...,And you removed it! You numbskull! I don't car...,1
4,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist",0


In [12]:
val_idxes = data.sample(int(data.shape[0] * 0.1)).index.values
train_idxes = list(set(np.arange(data.shape[0])) - set(val_idxes))

len(val_idxes) + len(train_idxes)

15410

In [13]:
train_data = data.loc[train_idxes].copy()
val_data = data.loc[val_idxes].copy()

train_data.reset_index(drop=True, inplace=True)
val_data.reset_index(drop=True, inplace=True)

In [19]:
train_data.to_csv("/Users/avgalichin/Documents/kaggle/Jigsaw2022/Datasets/train/jigsaw22_validation_data_swap.csv")
val_data.to_csv("/Users/avgalichin/Documents/kaggle/Jigsaw2022/Datasets/val/jigsaw22_validation_data_swap.csv")

In [5]:
data = Jigsaw22DatasetPaired([path], tokenizer, paired=True, batch_size=4, shuffle=False)
data.dataset

/Users/avgalichin/Documents/kaggle/Jigsaw2022/src/dataset.py:235: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, col_n] = df[col_n].str.replace('https?://\S+|www\.\S+', ' social medium ')
/Users/avgalichin/Documents/kaggle/Jigsaw2022/src/dataset.py:236: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, col_n] = df[col_n].str.replace('\s+', ' ')
/Users/avgalichin/Documents/kaggle/Jigsaw2022/src/data_utils.py:97: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(col_name, 1, inplace=True)


<PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 3), dtype=tf.int32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [28]:
data = data.drop_duplicates(subset=['less_toxic', 'more_toxic'])
data.reset_index(drop=True, inplace=True)
print(data.shape)
data.head()

(15410, 3)


,worker,less_toxic,more_toxic
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu..."
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist"


In [29]:
n_val = int(data.shape[0] * 0.1)
val_idxes = data.sample(n_val).index.values
train_idxes = list(set(np.arange(data.shape[0])) - set(val_idxes))

In [30]:
data_train = data.loc[train_idxes, :].reset_index(drop=True)
data_val = data.loc[val_idxes, :].reset_index(drop=True)

In [33]:
data_train.to_csv('/Users/avgalichin/Documents/kaggle/Jigsaw2022/Datasets/train/jigsaw22_validation_data.csv', index=False)
data_val.to_csv('/Users/avgalichin/Documents/kaggle/Jigsaw2022/Datasets/val/jigsaw22_validation_data.csv', index=False)

In [4]:
from src.dataset import Jigsaw22DatasetPaired
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [23]:
data = Jigsaw22DatasetPaired([path], tokenizer, paired=False, batch_size=4, shuffle=False)

/Users/avgalichin/Documents/kaggle/Jigsaw2022/src/dataset.py:236: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, col_n] = df[col_n].str.replace('https?://\S+|www\.\S+', ' social medium ')
/Users/avgalichin/Documents/kaggle/Jigsaw2022/src/dataset.py:237: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, col_n] = df[col_n].str.replace('\s+', ' ')
/Users/avgalichin/Documents/kaggle/Jigsaw2022/src/data_utils.py:97: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(col_name, 1, inplace=True)


In [19]:
model = _initialize_hugginface_pair_model('bert-base-cased', input_shape=(256, 3), n_classes=1)
loss_fn = MarginRankingLoss(margin=0.5)

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [34]:
for ex in data.dataset.take(5):
    pass

In [35]:
output = model(ex[0])

In [36]:
loss_fn(ex[1], output)

<tf.Tensor: shape=(), dtype=float32, numpy=0.41102004>

In [37]:
import torch

loss_torch = torch.nn.MarginRankingLoss(margin=0.5)

In [38]:
output

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[ 0.4514324 , -0.11756194],
       [ 0.38331032,  0.32938346],
       [-0.12125248, -0.1130318 ],
       [ 0.01265694, -0.19357073]], dtype=float32)>

In [39]:
loss_torch(
    torch.tensor(output.numpy()[:, 0]),
    torch.tensor(output.numpy()[:, 1]),
    torch.tensor(ex[1].numpy()))

tensor(0.4110)

In [2]:
data_df = pd.read_csv('/Users/avgalichin/Documents/kaggle/Jigsaw2022/Datasets/val/ruddit_data.csv')
print(data_df.shape)
data_df.head()

(570, 5)


,comment_id,label,label_group,input_ids,attention_mask
0,cza47xu,-0.062,7.0,"[0, 100, 218, 75, 206, 51, 214, 143, 430, 4, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,cza8437,0.042,9.0,"[0, 32523, 4, 125, 22, 19027, 1938, 113, 16, 4...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,cza8lst,-0.042,8.0,"[0, 3084, 4, 24839, 16, 7594, 14, 5, 2249, 11,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,czaeclj,0.083,10.0,"[0, 27847, 282, 75, 24, 67, 1266, 5, 1623, 280...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,czjeljo,0.104,10.0,"[0, 2709, 162, 16, 5, 1762, 9, 3031, 8066, 6, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [3]:
cfg = {
    'data_path': ['/Users/avgalichin/Documents/kaggle/Jigsaw2022/Datasets/train/pair_ruddit_data.csv'],
}
data = DummyJigsawDataset(**cfg)
data.dataset

<PrefetchDataset shapes: ((None, 128, 2), (None,)), types: (tf.int32, tf.float32)>

In [4]:
for ex in data.dataset.take(1):
    pass

In [5]:
dataset = data.dataset.unbatch()

input_lhs = dataset.map(
    lambda x, y: x)
input_rhs = dataset.map(
    lambda x, y: x)
input_data = tf.data.Dataset.zip((input_lhs, input_rhs))

input_data

<ZipDataset shapes: ((128, 2), (128, 2)), types: (tf.int32, tf.int32)>

In [7]:
for ex in input_data.take(1):
    ex, _ = tf.unstack(ex[0], axis=-1)

In [9]:
tf.shape(ex)[0]

<tf.Tensor: shape=(), dtype=int32, numpy=128>

In [2]:
cfg = {
    'data_path': ['/Users/avgalichin/Documents/kaggle/Jigsaw2022/Datasets/train/pair_ruddit_data.csv'],
    'max_length': 256,
    'pad_token': 1,
    'split_token': 2,
    'batch_size': 4,
    'n_epochs': None,
    'shuffle': True,
    'shuffle_buffer_size': int(100)
}

data = JigsawDatasetPaired(**cfg)
data.dataset

<PrefetchDataset shapes: ((None, 256, 2), (None,)), types: (tf.int32, tf.float32)>

In [4]:
for ex in data.dataset:
    pass

In [13]:
def insert(x):
    x[0] = 2
    return x

data_df.loc[:, 'input_ids'] = data_df.input_ids.apply(insert)

In [14]:
data_df.to_csv('/Users/avgalichin/Documents/kaggle/Jigsaw2022/Datasets/val/pair_ruddit_data.csv', index=False)

In [59]:
for i in data_df.index.values:
    cur_list = np.array(data_df.loc[i, 'input_ids'])
    new_list = cur_list[:np.where(cur_list == 1)[0][0]]
    
    data_df.loc[i, 'input_ids'] = new_list

ValueError: Must have equal len keys and value when setting with an iterable

In [57]:
data_df.loc[:, 'input_ids'] = data_df.input_ids.apply(lambda x: x[:np.where(np.array(x) == 1)[0][0]])
data_df.head()

IndexError: index 0 is out of bounds for axis 0 with size 0

In [22]:
tokenizer.decode(tokenizer('Fuck u', "Aboba")['input_ids'])

'<s>Fuck u</s></s>Aboba</s>'

In [14]:
tokenizer.eos_token

'</s>'

In [12]:
tokenizer.decode([1])

'<pad>'

In [11]:
tokenizer.encode('<s>')

[0, 0, 2]

In [4]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

In [2]:
data_dir = '/Users/avgalichin/Documents/kaggle/Jigsaw2022/Datasets/train/ub_data_sorted.csv'
data_df = pd.read_csv(data_dir)
print(data_df.shape)
data_df.head()

(310304, 4)


,id,label,input_ids,attention_mask
0,478544,0.020591,"[0, 25101, 8372, 8, 66, 12, 1116, 12, 6406, 8,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,337543,0.000000,"[0, 8346, 6, 38, 33, 7, 5170, 141, 24705, 473,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,5154914,0.020591,"[0, 133, 194, 11210, 3878, 7, 997, 7, 1597, 13...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,783447,0.020591,"[0, 113, 42301, 2857, 116, 166, 218, 75, 240, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,468970,0.000000,"[0, 24017, 4, 6142, 54, 4548, 41, 490, 7159, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [12]:
data_df.loc[:, 'input_ids'] = data_df.input_ids.apply(lambda x: list(map(int, x[1:-1].split(', '))))
data_df.loc[:, 'attention_mask'] = data_df.attention_mask.apply(lambda x: list(map(int, x[1:-1].split(', '))))
data_df.head()

,comment_id,label,label_group,input_ids,attention_mask
0,cza47xu,-0.062,7.0,"[0, 100, 218, 75, 206, 51, 214, 143, 430, 4, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,cza8437,0.042,9.0,"[0, 32523, 4, 125, 22, 19027, 1938, 113, 16, 4...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,cza8lst,-0.042,8.0,"[0, 3084, 4, 24839, 16, 7594, 14, 5, 2249, 11,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,czaeclj,0.083,10.0,"[0, 27847, 282, 75, 24, 67, 1266, 5, 1623, 280...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,czjeljo,0.104,10.0,"[0, 2709, 162, 16, 5, 1762, 9, 3031, 8066, 6, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [4]:
input_ids, attention_mask = np.stack(data_df.input_ids.values), np.stack(data_df.attention_mask.values)
scores = data_df.label.values

NameError: name 'np' is not defined

In [3]:
from src.dataset import JigsawDataset

In [6]:
data = JigsawDataset(
    data_dir,
    38788,
    32,
    shuffle=True)
dataset = data.dataset

In [7]:
iterator = dataset.as_numpy_iterator()

In [8]:
ex = next(iterator)

In [9]:
ex

(array([[[    0,     1],
         [ 2847,     1],
         [  745,     1],
         ...,
         [    1,     0],
         [    1,     0],
         [    1,     0]],
 
        [[    0,     1],
         [  133,     1],
         [  129,     1],
         ...,
         [    1,     0],
         [    1,     0],
         [    1,     0]],
 
        [[    0,     1],
         [11990,     1],
         [   47,     1],
         ...,
         [    1,     0],
         [    1,     0],
         [    1,     0]],
 
        ...,
 
        [[    0,     1],
         [20441,     1],
         [   35,     1],
         ...,
         [    1,     0],
         [    1,     0],
         [    1,     0]],
 
        [[    0,     1],
         [ 2847,     1],
         [   99,     1],
         ...,
         [    1,     0],
         [    1,     0],
         [    1,     0]],
 
        [[    0,     1],
         [  100,     1],
         [ 5170,     1],
         ...,
         [    1,     0],
         [    1,     0],
         [ 

In [ ]:
data = pd.read_csv('')